# COGS 118B - Final Project

# Names

- Abdul Ibrahim
- Kai Stern
- Patrick Helcl

# Abstract 

This study has the goal of utilizing more unique stock features to better cluster stock companies and their performance by using Gaussian Mixture Models instead of traditional K-means clustering methods. Our datasets were sourced from Kaggle rather than traditional financial sites for the added benefit of extra stock features (e.g stock splits) that are usually left out. This gives us the opportunity to try and identify more complex patterns that are unseen with simpler methods and features. We hope this project will present better investment opportunities by analyzing how the stocks are clustered together as well as identifying patterns that point towards growth. Our success will be measured by shedding light on the use of GMM in the stock analysis space by measuring the silhouette score to our baseline k-means model.

# Background

Stock choice and market prediction have been growing applications of machine learning (ML) over the last several decades. Recognizing and understanding patterns is crucial for both research and successful trading in a system as dynamic and complex as the stock market. Supervised learning approaches to stock market ML analysis typically require deep domain knowledge and extensive human contribution to perform feature engineering <a name="cite_ref-1"></a>[<sup>1</sup>](#cite_note-1). Automating this process frees up researchers to focus on other aspects of the problem and enables a lower domain knowledge threshold. Thus, there has been increasing interest in the use of unsupervised algorithms for financial ML purposes. Unsupervised clustering algorithms can help uncover unexpected structure within unlabelled data to suggest areas of research or give a competitive advantage for traders <a name="cite_ref-2"></a>[<sup>2</sup>](#cite_note-2). Principal components analysis (PCA) can help to group data based on the characteristics that contribute most to its variability <a name="cite_ref-3"></a>[<sup>3</sup>](#cite_note-3). Most research on the use of unsupervised learning for financial data is focused on the k-means clustering algorithm which has been found to be a good indicator of high and low volatility conditions across asset classes <a name="cite_ref-2"></a>[<sup>2</sup>](#cite_note-2). Multiple papers cited k-means’ poorer performance on messy data without spherical clusters as a limitation of the algorithm that could be overcome in future research by applying mixture models <a name="cite_ref-2"></a>[<sup>2</sup>](#cite_note-2) <a name="cite_ref-3"></a>[<sup>3</sup>](#cite_note-3) Other potential areas of research to avoid human feature engineering include self supervised learning so that algorithms can learn patterns from raw data themselves <a name="cite_ref-1"></a>[<sup>1</sup>](#cite_note-1). We plan to build upon the literature by implementing a Gaussian Mixture Model clustering that is more generalizable and unique relative to other studies, many of which use a k-means based approach in their research. 


# Problem Statement

After conducting background research on our topic of interest, we identified that the vast majority of machine learning literature frequently uses k-means clustering (a simple algorithm which assumes spherical/circular covariance) when conducting studies on stock data. Our group seeks to understand how clustering and pattern recognition will change when using a more robust and flexible algorithm. Gaussian mixture models are our current algorithm of choice as they can assume a wider array of covariance shape when conducting machine learning research on stock data. This enables us to cluster based on more irregular or less easily separable data. We hope to uncover new insights and better pattern recognition in our research. Our goal from this endeavor is to find potential worthwhile stock investments based on our clustering. 

# Data

### Dataset 1

Link to dataset 1 here: https://www.kaggle.com/datasets/iveeaten3223times/massive-yahoo-finance-dataset

Size: 9 Variables, 603K observations 

##### Features of each observation: 

- Date: The date corresponding to the recorded stock market data.
- Open: The opening price of the stock on a given date.
- High: The highest price of the stock reached during the trading day.
- Low: The lowest price of the stock observed during the trading day.
- Close: The closing price of the stock on a specific date.
- Volume: The volume of shares traded on the given date.
- Dividends: Any dividend payments made by the company on that date (if applicable).
- Stock Splits: Information regarding any stock splits occurring on that date.
- Company: Ticker symbol or identifier representing the respective company.

##### Critical values for this dataset

Since we only have 9 features and each of which is directly connected to the stock price we intend on using all 9 features listed above.

##### Summary of cleaning:

Since our data set has few features and is from Kaggle the values in our data are already very clean. However our group did find that the date values were stored as strings and were inconviently stored as the date and the time in a single string. To make this feature more useable we wrote a helper function to clean the string into just the date and convert it into datetime object, supported by pandas, for easier use later in the project with the date attributes and since knowing the exact times of the stock each day would not be necessary due to the timescale of our project.

##### Why this data is important for our project?:

This data is important for our project because ...

In [4]:
import torch
import pandas as pd
import numpy as np

df1 = pd.read_csv('data/stock_details_5_years.csv', header=0)

def time_fixer(x):
    x = x.split(' ')[0]
    return x

df1['Date'] = df1['Date'].apply(time_fixer)
df1['Date'] = pd.to_datetime(df1['Date'])

df1.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company
0,2018-11-29,43.829761,43.863354,42.639594,43.083508,167080000,0.00,0.0,AAPL
1,2018-11-29,104.769074,105.519257,103.534595,104.636131,28123200,0.00,0.0,MSFT
2,2018-11-29,54.176498,55.007500,54.099998,54.729000,31004000,0.00,0.0,GOOGL
3,2018-11-29,83.749496,84.499496,82.616501,83.678497,132264000,0.00,0.0,AMZN
4,2018-11-29,39.692784,40.064904,38.735195,39.037853,54917200,0.04,0.0,NVDA


# Data 2

Dataframe created from the first data set. *full explanation somewhere here*. We are created a data frame from a subset of the data so that we can group the different stocks by their volatility and run our unsupervised learning / gmm model on the open and close data and leverage these features in the most informational way.

In [6]:
df1.head()

df1['Difference'] = df1['Open'] - df1['Close']

df_diff_arrays = df1.groupby('Company')['Difference'].apply(np.array).reset_index()

df_diff_arrays['Normalized'] = df_diff_arrays['Difference'].apply(lambda x: x / np.linalg.norm(x))

df_diff_arrays.head()

,Company,Difference,Normalized
0,A,"[-0.3277933886396909, -0.6941591611439009, -1....","[-0.005529169310739379, -0.011708971759599146,..."
1,AAPL,"[0.7462530351512058, 0.41031706833509674, -0.0...","[0.010935155997460077, 0.0060125466019118265, ..."
2,ABBV,"[-0.7125628771194101, -3.174853337607999, 0.52...","[-0.01543955809048262, -0.06879158893727623, 0..."
3,ABEV,"[-0.025338456560350053, -0.00844588268194002, ...","[-0.013505524424249502, -0.004501697827316716,..."
4,ABNB,"[1.2899932861330115, 7.3000030517580115, 5.0, ...","[0.01100236923219863, 0.0622618193714668, 0.04..."


# Proposed Solution

After doing research on current literature in financial analysis using machine learning for the stock market, our group found that there was an overabundance of k-means in studies. This over use of k-means for clustering analysis and model selection was even addressed as a marker for need of further research in the subject by <a name="cite_ref-3"></a>[<sup>3</sup>](#cite_note-3). Our group seeks to expand to the current literature by using a Gaussian mixture model GMM for clustering of different attributes in the dataset, which can assume covariances other than spherical/circular unlike k-means which have fixed covariance assumptions. We aim to find new and/or more accurate clusterings and patterns than in previous research which could lead to more accurate analysis of stock performance. To determine if the GMM performs better than k-means, our group will also implement a k-means model trained on our data as a benchmark comparison for our findings and metrics. One such metric that will be used to compare the two is each model's Silhouette score. The silhouette score measures how well each cluster is separated from one another and each cluster’s density of data points. Comparing the silhouette score of each model will be a good measure of how well our GMM is performing relative to the k-means model, also giving our group an idea of how well our model stacks up to the models used in our background research articles. Our group also plans to use k-folds cross validation for hyperparameter tuning to ensure the best versions of each model are being compared. Our methodology should be highly reproducible especially if the same years of stock market data are used. Whether or not better metrics are achieved by our model compared to the k-means model, our project will shed light on ML based stock market analysis literature. This will be achieved by showing that our GMM out performed k-means inviting further research into the use of GMM in the space or alternately ruling out the use of GMM’s in further research and supporting the classic k-means and suggesting research into other models beyond the two are needed.

# Evaluation Metrics

We plan on using Silhouette score to evaluate our clusterings. This metric measures how well separated and dense clusters are. It returns a value between -1 and 1. Negative values suggest data points have been placed in the wrong clusters as data points are closer to points in other clusters. Values near 0 suggest highly overlapping clusters. Values near 1 suggest well separated clusters, dense clusters. Values near 1 are better for Silhouette score. Silhouette scores can break down if clusters are highly irregular. If that ends up being the case with our data we will supplement how we evaluate our models with other evaluation metrics such as adjusted rand score.


# Results

You may have done tons of work on this. Not all of it belongs here. 

Reports should have a __narrative__. Once you've looked through all your results over the quarter, decide on one main point and 2-4 secondary points you want us to understand. Include the detailed code and analysis results of those points only; you should spend more time/code/plots on your main point than the others.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Subsection 1

You will likely have different subsections as you go through your report. For instance you might start with an analysis of the dataset/problem and from there you might be able to draw out the kinds of algorithms that are / aren't appropriate to tackle the solution.  Or something else completely if this isn't the way your project works.

### Subsection 2

Another likely section is if you are doing any feature selection through cross-validation or hand-design/validation of features/transformations of the data

### Subsection 3

Probably you need to describe the base model and demonstrate its performance.  Maybe you include a learning curve to show whether you have enough data to do train/validate/test split or have to go to k-folds or LOOCV or ???

### Subsection 4

Perhaps some exploration of the model selection (hyper-parameters) or algorithm selection task. Validation curves, plots showing the variability of perfromance across folds of the cross-validation, etc. If you're doing one, the outcome of the null hypothesis test or parsimony principle check to show how you are selecting the best model.

### Subsection 5 

Maybe you do model selection again, but using a different kind of metric than before?



# Discussion

### Interpreting the result

OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.

### Limitations

Are there any problems with the work?  For instance would more data change the nature of the problem? Would it be good to explore more hyperparams than you had time for?   



### Ethics & Privacy

In regards to ethics and privacy our group’s project does not have any major ethical or privacy related concerns. Our project is built on data from publicly traded stocks of companies, which as the name implies are freely available to the public to access. 

Additionally, our data set does not contain any information about individuals since all data pertains to different companies, meaning we do not have to account for or worry about precautions such as personally identifiable data or major biases in data (e.g. political or socio economic). Additional benefits of using publicly available data on business and corporations instead of data from human beings include no need for informed consent, confidentiality and/or anonymity of participants. 

Since the data being used is from Kaggle, one potential issue with our data is the potential for altered or fabricated data about the different stocks and companies during scraping from the original poster. Some examples of this would include: errors in the data aggregation process or code that would store incorrect data, neglect of certain time periods or companies for sake of efficiency or information size, and personal bias from the data curator to select companies from yahoo finance based on some reason undisclosed on Kaggle (therefore unavailable for our group to identify). If any issues with the data are identified at any point in this project our group will take the necessary steps and precautions to identify, eliminate, or correct these issues and concerns.

Another potential concern as a result of the project is potential for our findings to influence stock investors which could theoretically have effects on the trading volume of different stocks or stocks with certain attributes identified with our models. In this scenario, the company's stock price would either be increased or decreased from our findings. However, it is worth noting that with our group member’s background and no previous experience in analyzing financial data we find this scenario to be extremely unlikely since our project should be viewed as a non reputable source and not comparable to suggestions from professional financial organizations. That being said, it is our group’s responsibility to identify any possible concerns and risks of the project no matter how likely. 

Overall this project has very few concerns and issues which also happen to be very unlikely after all things considered, thus this project should be free of any major confounds and issues that would make the accuracy and applicability of our findings questionable.



### Conclusion

Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.

# Team Expectations 

* *Schedule meetings and address concerns via text group chat*
* *Meet via zoom*
* *Break up work evenly; don't let one person carry the entire project*
* *Complete individual tasks in a timely manner*
* *Be open about strengths and weaknesses for project goals. Don't be afraid to ask for help*
* *Be open minded about other's ideas and have willingness to listen*

# Project Timeline Proposal


| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 2/8  |  2:10 PM |  Brainstorm topics/questions (all)  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 2/17 |  2 PM |  Finalize research question (all) | Discuss ideal dataset(s) and ethics; draft project proposal | Divide proposal work to individual pieces |
| 2/19  | 3:30 PM  | Work on assigned parts | Combine individual parts, Finalize project structure, Complete and turn in  project proposal |
| 2/20  | 3:00 PM  | Polish proposal and submit | Changes to be made to proposal, start brainstorm for final project notebook/sections to add |
| 2/24  | 2 PM  | Start final jupyter notebook | Structure foundation of final project notebook, EDA for dataset. Plan overall workflow for coding segments of project |
| 3/2  | 2 PM  | Re-assign tasks to each group members for contributions | Move to different part of project according to which parts are finished or need work still |
| 3/9  | 2 PM  | Finish first full draft of final project | Collectively review individual sections, Discuss/edit full project, go over cohesivness of sections |
| 3/16  | 2 PM  | Ask TA quick review (if needed/time allows) | Address TA suggestions, One final review as group and implement changes from criticism  |
| 3/18  | 6:00 PM  | Finalize changes to project and submit |  Any last minute changes or ideas before submitting |

# Footnotes

1. <a name="cite_note-1"></a> [^](#cite_ref-1) Ying, Z. et al. (1 Feb 2024) Predicting stock market trends with self-supervised learning and extreme value theory. *Neurocomputing*. https://www.sciencedirect.com/science/article/abs/pii/S0925231223011566

2. <a name="cite_note-2"></a> [^](#cite_ref-2) Barradas, A., et al. (July, 2023) Identification of Patterns in the Stock Market through Unsupervised Algorithms. *MDPI*. https://www.mdpi.com/2813-2203/2/3/33

3. <a name="cite_note-3"></a> [^](#cite_ref-3) Lee, J. (1 Sep 2016) Clustering Approaches for Financial Data Analysis: a Survey. *School of Computer Science Science & Informatics, University College Dublin, Ireland*. https://arxiv.org/abs/1609.08520
